In [1]:
from knmi.downloader import KNMIDownloader
from knmi.database import KNMIDatabase
from knmi.processor import KNMIProcessor
from config import xlBasisfile, dbFile 
import pandas as pd

# Instantieer classes
db = KNMIDatabase(dbFile, xlBasisfile)
    
downloader = KNMIDownloader(db)
processor = KNMIProcessor(db)

# als de opgegeven database-file niet bestaat, wordt een nieuwe database aangemaakt
if not db.exists():
    db.create_database()
    db.fill_database()

# de database wordt (aan)gevuld met de meest recente data
#downloader.update_data()

#db.get_last_observations


In [2]:
processor.set_filter(locations= ['De Bilt'], startyear= 2020, endyear=2022,startmonth=1,endmonth=12)

dfdp = processor.get_filtered_PDseries(bottom=0)

P = db.get_station_timeseries(1,'P','2020-01-01','2022-12-31')
E = db.get_station_timeseries(16,'E','2020-01-01','2022-12-31')

df_check = P.merge(E, left_index=True, right_index=True, suffixes=('_P','_E')).merge(dfdp.rename('dfdp'), left_index=True, right_index=True)

df_check.to_clipboard()
#dfdp.rename('dfdp')

Processing location: De Bilt


In [7]:
locations = ['Groningen', 'De Bilt']

locs = db.get_locations()
if locations == 'all':
    locs = locs.index
elif isinstance(locations, list):
            locs = locs.loc[locs['LocationName'].isin(locations)].index
elif isinstance(locations, str):
            locs = locs.loc[locs['LocationName'] == locations].index

locs

Index([1, 3], dtype='int64', name='LocationId')

In [8]:
processor.filter_locations()

'Groningen'

## Snelheid testen

In [3]:
import sqlite3
import timeit

qry = '''SELECT date(Timestamp) AS date, value, quality 
                             FROM KNMI_data 
                             WHERE StationId = ? AND Parameter = ? AND Timestamp BETWEEN ? AND ?;
                       '''

with sqlite3.connect(db.db_path) as conn:
            
            startdate = pd.to_datetime('1900-01-01').to_julian_date()
            enddate = pd.to_datetime('2020-12-31').to_julian_date()

            cursor = conn.cursor()
            t0 = timeit.default_timer()
            result1 = cursor.execute(qry, (1,'P',startdate, enddate)).fetchall()
            t1 = timeit.default_timer()
            result2 = pd.read_sql_query(qry,conn, params= (1,'P',startdate,enddate))
            t2 = timeit.default_timer()

            print(f'Time 1: {t1-t0}')
            print(f'Time 2: {t2-t1}')
            

Time 1: 0.09764350000000377
Time 2: 0.10066589999999564


### 

In [29]:
qry2 = '''SELECT date(Timestamp) AS date, StationId, Parameter, value, quality 
                             FROM KNMI_data 
                             WHERE Timestamp BETWEEN ? AND ?;
                       '''

with sqlite3.connect(db.db_path) as conn:
            
            startdate = pd.to_datetime('1900-01-01').to_julian_date()
            enddate = pd.to_datetime('2020-12-31').to_julian_date()

            cursor = conn.cursor()
            t0 = timeit.default_timer()
            result1 = cursor.execute(qry2, (startdate, enddate)).fetchall()
            t1 = timeit.default_timer()
            result2 = pd.read_sql_query(qry2,conn, params = (startdate,enddate))
            t2 = timeit.default_timer()

            print(f'Time 1: {t1-t0}')
            print(f'Time 2: {t2-t1}')


Time 1: 1.0279921999999715
Time 2: 1.1640588000000207


In [7]:
import sqlite3

#with sqlite3.connect(db.db_path) as conn:
res3 = db.get_location_stations()

res3

,StationId,Parameter,valid_from,valid_through,first_date,last_date,obs_from,obs_through
0,1,P,1-1-1906,None,1898-01-01,2025-12-20,1906-01-01,2025-12-20
1,16,E,1-1-1901,None,1957-07-01,2026-01-04,1957-07-01,2026-01-04
0,2,P,1-1-1973,None,1958-04-01,2025-12-20,1973-01-01,2025-12-20
1,3,P,1-1-1906,31-12-1972,1851-01-01,1972-12-31,1906-01-01,1972-12-31
2,16,E,1-1-1901,31-12-2000,1957-07-01,2026-01-04,1957-07-01,2000-12-31
3,17,E,1-1-2001,None,1964-11-01,2026-01-04,2001-01-01,2026-01-04
0,4,P,1-1-1906,None,1847-01-01,2025-12-20,1906-01-01,2025-12-20
1,16,E,1-1-1901,31-12-2000,1957-07-01,2026-01-04,1957-07-01,2000-12-31
2,18,E,1-1-2001,None,1965-01-01,2026-01-04,2001-01-01,2026-01-04
0,5,P,1-1-1906,None,1892-08-31,2025-12-20,1906-01-01,2025-12-20


# Complexe(re) queries

In [8]:
qry3 =  '''SELECT StationId, Name, KNMI_data.Parameter as Param, date(min(Timestamp)) AS first_date, date(max(Timestamp)) AS last_date 
                            FROM KNMI_data INNER JOIN KNMI_stations USING (StationId)
                            GROUP BY Param, Name;
                     '''
with sqlite3.connect(db.db_path) as conn:
    res3 = pd.read_sql_query(qry3,conn)

res3.set_index('StationId').sort_index()

,Name,Param,first_date,last_date
StationId,,,,
1,De Bilt,P,1898-01-01,2025-12-20
2,De Kooy,P,1958-04-01,2025-12-20
3,Den Helder,P,1851-01-01,1972-12-31
4,Groningen,P,1847-01-01,2025-12-20
5,Heerde,P,1892-08-31,2025-12-20
6,Hoofddorp,P,1866-12-01,2025-12-20
7,Hoorn,P,1883-04-01,2025-12-20
8,Kerkwerve,P,1878-04-02,2025-12-20
9,Oudenbosch,P,1888-02-02,2025-12-20


In [4]:
with sqlite3.connect(db.db_path) as conn:
    res3 = pd.read_sql_query(db.qry_stations,conn)

res3.set_index('StationId').sort_index()

,Name,Parameter,Code,Url,FileTypeId,Timestep
StationId,,,,,,
1,De Bilt,P,550,https://cdn.knmi.nl/knmi/map/page/klimatologie...,1,1d
2,De Kooy,P,25,https://cdn.knmi.nl/knmi/map/page/klimatologie...,1,1d
3,Den Helder,P,9,https://cdn.knmi.nl/knmi/map/page/klimatologie...,1,1d
4,Groningen,P,139,https://cdn.knmi.nl/knmi/map/page/klimatologie...,1,1d
5,Heerde,P,328,https://cdn.knmi.nl/knmi/map/page/klimatologie...,1,1d
6,Hoofddorp,P,438,https://cdn.knmi.nl/knmi/map/page/klimatologie...,1,1d
7,Hoorn,P,222,https://cdn.knmi.nl/knmi/map/page/klimatologie...,1,1d
8,Kerkwerve,P,737,https://cdn.knmi.nl/knmi/map/page/klimatologie...,1,1d
9,Oudenbosch,P,828,https://cdn.knmi.nl/knmi/map/page/klimatologie...,1,1d


In [11]:
#qry4 =  '''SELECT LocationName, KNMI_data.Parameter , StationId, date(min(Timestamp)) AS first_date, date(max(Timestamp)) AS last_date
#                            FROM KNMI_data 
#                              RIGHT JOIN (P13_Locations RIGHT JOIN locations_stations USING (LocationId))
#                             USING (StationID)
#                            ;
#                     '''

qry4 =  '''SELECT LocationId, locations_stations.StationId, date(min(Timestamp)) AS first_date, date(max(Timestamp)) AS last_date
                            FROM locations_stations 
                              INNER JOIN KNMI_data ON locations_stations.StationId = KNMI_data.StationId
                              GROUP BY LocationId, locations_stations.StationId;'''

qry5 = '''SELECT LocationName, locations_stations.LocationId, locations_stations.StationId,locations_stations.Parameter, locations_stations.valid_from, locations_stations.valid_through ,date(min(Timestamp)) AS first_date, date(max(Timestamp)) AS last_date 
          FROM P13_locations
           INNER JOIN (locations_stations INNER JOIN KNMI_data ON locations_stations.Parameter = KNMI_data.Parameter AND locations_stations.StationId=KNMI_data.StationId) B
            ON P13_locations.LocationId = B.LocationId
          GROUP BY P13_locations.LocationId, B.Parameter;'''

with sqlite3.connect(db.db_path) as conn:
    res4 = pd.read_sql_query(qry5,conn)

res4

,LocationName,LocationId,StationId,Parameter,valid_from,valid_through,first_date,last_date
0,De Bilt,1,16,E,1-1-1901,None,1957-07-01,2026-01-04
1,De Bilt,1,1,P,1-1-1906,None,1898-01-01,2025-12-20
2,De Kooy,2,16,E,1-1-1901,31-12-2000,1957-07-01,2026-01-04
3,De Kooy,2,2,P,1-1-1973,None,1851-01-01,2025-12-20
4,Groningen,3,16,E,1-1-1901,31-12-2000,1957-07-01,2026-01-04
5,Groningen,3,4,P,1-1-1906,None,1847-01-01,2025-12-20
6,Heerde,4,16,E,1-1-1901,31-12-2000,1957-07-01,2026-01-04
7,Heerde,4,5,P,1-1-1906,None,1892-08-31,2025-12-20
8,Hoofddorp,5,16,E,1-1-1901,31-12-2000,1957-07-01,2026-01-04
9,Hoofddorp,5,6,P,1-1-1906,None,1866-12-01,2025-12-20


In [15]:
qry6 = '''SELECT locations_stations.*, date(min(Timestamp)) AS first_date, date(max(Timestamp)) AS last_date 
          FROM locations_stations 
          INNER JOIN KNMI_data ON locations_stations.Parameter = KNMI_data.Parameter AND locations_stations.StationId=KNMI_data.StationId
          GROUP BY locations_stations.LocationId, locations_stations.StationId, locations_stations.Parameter;'''

qry7 = "SELECT * FROM locations_stations;"

with sqlite3.connect(db.db_path) as conn:
    res6 = pd.read_sql_query(qry6,conn)

res6

,LocationStatId,LocationId,Parameter,StationId,valid_from,valid_through,first_date,last_date
0,1,1,P,1,1-1-1906,None,1898-01-01,2025-12-20
1,16,1,E,16,1-1-1901,None,1957-07-01,2026-01-04
2,2,2,P,2,1-1-1973,None,1958-04-01,2025-12-20
3,3,2,P,3,1-1-1906,31-12-1972,1851-01-01,1972-12-31
4,17,2,E,16,1-1-1901,31-12-2000,1957-07-01,2026-01-04
5,18,2,E,17,1-1-2001,None,1964-11-01,2026-01-04
6,4,3,P,4,1-1-1906,None,1847-01-01,2025-12-20
7,19,3,E,16,1-1-1901,31-12-2000,1957-07-01,2026-01-04
8,20,3,E,18,1-1-2001,None,1965-01-01,2026-01-04
9,5,4,P,5,1-1-1906,None,1892-08-31,2025-12-20


## Query location_stations testen

In [8]:
import sqlite3
import timeit


t0 = timeit.default_timer()
result1 = db.get_location_stations('De Kooy')
t1 = timeit.default_timer()
result2 = db.get_location_stations2('De Kooy')
t2 = timeit.default_timer()

print(f'Time 1: {t1-t0}')
print(f'Time 2: {t2-t1}')

Time 1: 0.3789833000000158
Time 2: 1.518287799999996


In [9]:
result1

,StationId,Parameter,valid_from,valid_through,first_date,last_date,obs_from,obs_through
0,2,P,1-1-1973,None,1958-04-01,2025-12-20,1973-01-01,2025-12-20
1,3,P,1-1-1906,31-12-1972,1851-01-01,1972-12-31,1906-01-01,1972-12-31
2,16,E,1-1-1901,31-12-2000,1957-07-01,2026-01-04,1957-07-01,2000-12-31
3,17,E,1-1-2001,None,1964-11-01,2026-01-04,2001-01-01,2026-01-04


In [10]:
result2

,LocationStatId,LocationId,Parameter,StationId,valid_from,valid_through,first_date,last_date
2,2,2,P,2,1-1-1973,None,1958-04-01,2025-12-20
3,3,2,P,3,1-1-1906,31-12-1972,1851-01-01,1972-12-31
4,17,2,E,16,1-1-1901,31-12-2000,1957-07-01,2026-01-04
5,18,2,E,17,1-1-2001,None,1964-11-01,2026-01-04
